Inspired by a course Using Databases with Python on Coursera, I decided to parse information of places I am interested in using the Google GeoCoding API and store them in an MS Access file.

I chose Access 2013 simply because I was practicing on a computer in the library which has no other type of database installed. However, there does not exist an equivalent to MySQL 'CREATE TABLE IF NOT EXISTS' in MS Access which is a piece of useful code I found in the sample code of this course.

In [ ]:
import urllib.parse
import urllib.request
import os
import pyodbc
import json
import time
import ssl

In [ ]:
serviceurl = "http://maps.googleapis.com/maps/api/geocode/json?"

dataFile = "geodata.accdb"
databaseFile = os.getcwd() + "\\" + dataFile
connectionString = "Driver={Microsoft Access Driver (*.mdb, *.accdb)}; Dbq=%s" % databaseFile
conn = pyodbc.connect(connectionString)
cur = conn.cursor()

cur.execute('''
CREATE TABLE Locations (address TEXT, geodata TEXT)''')

fh = open("where.data")
count = 0
for line in fh:
    if count > 200 : break
    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?", (address, ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    print('Resolving', address)
    url = serviceurl + urllib.parse.urlencode({"sensor":"false", "address": address})
    print('Retrieving', url)
    uh = urllib.request.urlopen(url)
    data = uh.read()
    count += 1
    try: 
        js = json.loads(str(data))
    except: 
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') : 
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata) 
            VALUES ( ?, ? )''', ( address,data ) )
    conn.commit() 
    time.sleep(2)